__Imports__

In [2]:
import torch
import torch.nn as nn
import scipy.stats as stats
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchviz import make_dot

from icnnet import ICNNet
from mydataset import MyDataset


__Toy data TR 1D__


In [2]:
n = 1000

x = torch.concat([torch.tensor(stats.norm.rvs(loc=-1, scale=0.1, size=(n,1)), dtype=torch.float32), torch.tensor(stats.norm.rvs(loc=1, scale=0.1, size=(n,1)), dtype=torch.float32)])
#y = torch.concat([torch.zeros(n//2), torch.ones(n//2), torch.zeros(n//2), torch.ones(n//2)])
#c = torch.concat([torch.zeros(n//2), torch.ones(n//2), torch.zeros(n//2), torch.ones(n//2)])

y = torch.concat([-torch.ones(n), torch.ones(n)])
c = torch.concat([torch.ones(2*n)])

X = torch.stack(torch.unbind(x,dim=0))
Y = torch.stack(torch.unbind(y,dim=0)).unsqueeze(1)
C = torch.stack(torch.unbind(c,dim=0)).unsqueeze(1)

print(X)


tensor([[-1.0661],
        [-1.0001],
        [-0.9680],
        ...,
        [ 0.9911],
        [ 1.0228],
        [ 0.8575]])


__Toy data valou 1D__

In [3]:
n = 1000

x = torch.concat([torch.tensor(stats.norm.rvs(loc=-1, scale=1, size=(2*n,1)), dtype=torch.float32)])
#y = torch.concat([torch.zeros(n//2), torch.ones(n//2), torch.zeros(n//2), torch.ones(n//2)])
#c = torch.concat([torch.zeros(n//2), torch.ones(n//2), torch.zeros(n//2), torch.ones(n//2)])

c = torch.concat([-torch.ones(n), torch.ones(n)])

X = torch.stack(torch.unbind(x,dim=0))
C = torch.stack(torch.unbind(c,dim=0)).unsqueeze(1)
C = C[torch.randperm(C.size(0))]

Y = C*X
print(X)
print(C)
print(Y)



tensor([[-1.5050],
        [-0.7328],
        [-0.7697],
        ...,
        [-0.4425],
        [-1.0191],
        [-2.0095]])
tensor([[ 1.],
        [ 1.],
        [ 1.],
        ...,
        [-1.],
        [ 1.],
        [-1.]])
tensor([[-1.5050],
        [-0.7328],
        [-0.7697],
        ...,
        [ 0.4425],
        [-1.0191],
        [ 2.0095]])


__Toy data Gaussian__

In [67]:
d = 2 # dimension of the data
r = 100 # number of points to simulate my gaussian
N = 1000 # number of samples


# simulate the gaussian
x = torch.tensor(stats.norm.rvs(loc=0, scale=1, size=(N, r, d)), dtype=torch.float32)
locs = torch.randint(-10, 10, (N,), dtype=torch.float32).repeat_interleave(r).view(N, r)
scales = 5*torch.rand(N, dtype=torch.float32).repeat_interleave(r).view(N, r)

c = torch.stack((locs, scales), dim = 2)

locs2D = locs.unsqueeze(-1).expand(-1, -1, d)
scales2D = scales.unsqueeze(-1).expand(-1, -1, d)

y = torch.tensor(stats.norm.rvs(loc=locs2D, scale=scales2D, size=(N, r, d)), dtype=torch.float32)

print(x.size())
print(c.size())
print(y.size())

torch.Size([1000, 100, 2])
torch.Size([1000, 100, 2])
torch.Size([1000, 100, 2])


__Loading the dataset__

In [68]:


dataset = MyDataset(x, c, y)

batch_size = 100  # Example batch size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

__PICNN training__

In [69]:
from torch import optim
epochs = 50

# Initialize the model
model = ICNNet()
model.train()

# Define the loss function and the optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(epochs):
    for x_batch, c_batch, y_batch in dataloader:
        x_batch.requires_grad_(True)
        c_batch.requires_grad_(True)

        optimizer.zero_grad() # Zero the gradients
        
        output = model(x_batch, c_batch)  # Assuming context c is same as input x

        loss = criterion(output, y_batch) # Compute the loss
        loss.backward() # Backward pass

        optimizer.step() # Update the parameters
        # for layers_k in model.layers_z:
        #     for param in layers_k.parameters():
        #         param.data.clamp_min_(0)
        
        #pass

    #for name, parameter in model.named_parameters():
        #if parameter.requires_grad and parameter.grad is not None:
            #grad_norm = parameter.grad.norm().item()
            #print(f"Gradient norm for {name}: {grad_norm}")

    print(f"Epoch {epoch+1}/{epochs} Loss: {loss.item()}")




RuntimeError: mat1 and mat2 shapes cannot be multiplied (10000x2 and 1x1)

In [12]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for x_batch, c_batch, y_batch in dataloader:
        outputs = model(x_batch, c_batch)
        for result, y in zip(outputs, y_batch) :
            print(result, y)
        break



tensor([0.9659]) tensor([0.9711])
tensor([-0.8608]) tensor([-0.8804])
tensor([-1.5514]) tensor([-1.5494])
tensor([-2.0634]) tensor([-2.0654])
tensor([-0.3589]) tensor([-0.3625])
tensor([-0.0569]) tensor([-0.0557])
tensor([0.2998]) tensor([0.3047])
tensor([0.5297]) tensor([0.5354])
tensor([-0.1235]) tensor([-0.1228])
tensor([0.9448]) tensor([0.9500])
tensor([1.9734]) tensor([1.9713])
tensor([-0.3519]) tensor([-0.3491])
tensor([1.6254]) tensor([1.6264])
tensor([2.1131]) tensor([2.1097])
tensor([1.3164]) tensor([1.3198])
tensor([2.5679]) tensor([2.5597])
tensor([0.2010]) tensor([0.2053])
tensor([1.0198]) tensor([1.0247])
tensor([0.7091]) tensor([0.7148])
tensor([-0.9824]) tensor([-0.9785])
tensor([1.9971]) tensor([1.9948])
tensor([0.9218]) tensor([0.9271])
tensor([0.4286]) tensor([0.4341])
tensor([2.5955]) tensor([2.5870])
tensor([1.0804]) tensor([1.0850])
tensor([-0.4405]) tensor([-0.4374])
tensor([0.6309]) tensor([0.6259])
tensor([-1.2735]) tensor([-1.2702])
tensor([-1.1432]) tensor([-1

__Makkuva training__

In [14]:
from torch import optim
epochs = 100
train_freq_g = 20

# Initialize the model
input_size = 1
ICNNf = ICNNet(layer_sizes = [input_size, 2,2,2,2,1], context_layer_sizes=[1,2,2,2,2,2])

output_size = 1
ICNNg = ICNNet(layer_sizes = [output_size, 2,2,2,2,1], context_layer_sizes=[1,2,2,2,2,2])

# Define the loss function and the optimizer
optimizer_f = optim.Adam(ICNNf.parameters())
optimizer_g = optim.Adam(ICNNg.parameters())

for epoch in range(epochs):
   
    for _ in range(train_freq_g) :
        for x, c, y in dataloader:
            optimizer_f.zero_grad() # Zero the gradients
            optimizer_g.zero_grad() # Zero the gradients
            x.requires_grad_(True)
            y.requires_grad_(True)
            c.requires_grad_(True)

            output_g = ICNNg(y, c)
            diff_output_g = torch.autograd.grad(output_g, y, grad_outputs=torch.ones_like(output_g), create_graph=True)[0]
            
            loss_g = - torch.matmul(y.transpose(0, 1), diff_output_g) - ICNNf(diff_output_g, c)
            loss_g = torch.mean(loss_g)
            loss_g.backward() # Backward pass

            optimizer_g.step() # Update the parameters
            for layers_k in ICNNg.layers_z:
                for param in layers_k.parameters():
                    param.data.clamp_min_(0)

    for x, c, y in dataloader:
        optimizer_f.zero_grad() # Zero the gradients
        optimizer_g.zero_grad()
        x.requires_grad_(True)
        y.requires_grad_(True)
        c.requires_grad_(True)
        


        output_g = ICNNg(y, c)
        diff_output_g = torch.autograd.grad(outputs=output_g, inputs=y, grad_outputs=torch.ones_like(output_g), create_graph=True)[0]

        loss_f = ICNNf(x, c) - ICNNf(diff_output_g, c)
        loss_f = torch.mean(loss_f)
        loss_f.backward() # Backward pass
        
        for layers_k in ICNNf.layers_z:
                for param in layers_k.parameters():
                    param.data.clamp_min_(0)


        #testing algorithm
        output_g_test = ICNNg(x, c)
        diff_output_g_test = torch.autograd.grad(output_g_test, x, grad_outputs=torch.ones_like(output_g_test), create_graph=True)[0]

        loss_test = diff_output_g_test-y

        #print('y =', y[0], 'diff_output_g =', diff_output_g_test[0])
        
        pass

    print(f"Epoch {epoch+1}/{epochs} loss_g: {loss_g.item()}, loss_f: {loss_f.item()}")

Epoch 1/100 loss_g: -14697.9033203125, loss_f: -26.161470413208008
Epoch 2/100 loss_g: -13241685114880.0, loss_f: -23073417216.0
Epoch 3/100 loss_g: -4.9078584419444326e+17, loss_f: -840131305013248.0


KeyboardInterrupt: 